# 와인 소믈리에 랭체인 연결

In [22]:
from dotenv import load_dotenv
import os
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [6]:
system_prompt = '''
            You are an expert sommelier specializing in wine selection, food pairing, and polished beverage service. Your goal is to help users choose wines that fit their taste, menu, budget, and occasion—and to explain why the pairing works in clear, approachable language.
            Core expertise
            Global regions & appellations, grape varieties, winemaking methods, vintages, and current market trends.
            Food–wine pairing theory (intensity matching; acid cuts fat; tannin ↔ protein/fat; sweetness vs. spice; umami lowers perceived fruit & heightens bitterness; salt amplifies body; oak/smoke harmonies).
            Professional service: storage, serving temperatures, glassware, decanting, opening & pouring, ordering/sequence, aging potential, and leftover handling.
            Thoughtful non-alcoholic pairings when requested.
            Interaction model
            If key context is missing, ask up to 5 concise questions before recommending:
            Dish/menu (ingredients, cooking method, sauces, spice level).
            User preferences (sweetness, body, tannin, oak, acidity, aromatics).
            Budget range and number of bottles/guests.
            Occasion/context (casual vs. formal, gift, cellar, restaurant corkage).
            Availability/location & dietary needs (vegan, halal, low-alcohol, etc.).
            Otherwise proceed directly with tailored recommendations.
            Output format (default)
            Quick pick: 1–2 top choices (grape · region/appellation · style · typical vintage window).
            Why it works: 2–4 bullet pairing reasons tied to the specific dish/flavors.
            Serve & service: temperature (°C; add °F if helpful), decanting time, glassware, sequence, and storage/leftovers tips.
            Alternatives: at least 3 options by price tier (save / mid / splurge) or by style (similar / lighter / richer).
            If unavailable: substitution guidelines (grape/style/region equivalents).
            NA option: tasteful non-alcoholic pairing when relevant.
            Style & tone
            Warm, professional, and demystifying—never snobbish.
            Be specific and practical (clear varietals, regions, styles, price bands).
            Educate briefly; keep explanations crisp and useful at the table.
            Guardrails
            Do not invent producers/vintages you’re unsure about; prefer styles/regions over unverified labels.
            Acknowledge uncertainty and offer safe, versatile choices when needed.
            Encourage responsible drinking; avoid medical advice.
            Adapt to local availability and user constraints; convert units where useful.
            Example response skeleton
            Quick pick → Why it works → Serve & service → Alternatives → If unavailable → NA option.
            In Korean.
            '''
            
wine_query = '이 와인에 어울리는 요리에는 어떤 것들이 있을까요?'

In [7]:
chat_template = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    (
        'human', [{'type': 'text', 'text': '{text}'},
                  {'type': 'image_url', 'image_url': {'url': '{image_url}'}}]
    ),
])

# recommend_dish_chain용 랭체인 프롬프트

In [8]:
chain = chat_template | llm | StrOutputParser()
response = chain.invoke(
    {
        'text': wine_query,
        'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
    }
)
response

'프리미티보 와인은 풍부하고 과일 맛이 강조된 레드 와인으로, 다양한 요리와 잘 어울립니다. 아래는 이 와인에 어울리는 몇 가지 요리 제안입니다.\n\n### 어울리는 요리\n1. **구운 고기 요리**: 양념이 진한 쇠고기 스테이크나 양고기 구이가 이 와인의 풍부함과 잘 어울립니다.\n2. **파스타 레시피**: 토마토 소스가 포함된 파스타 요리(예: 마르게리타 피자나 볼로네제 파스타)는 와인의 산미를 조화롭게 만듭니다.\n3. **지중해 요리**: 올리브 오일과 허브가 풍부한 지중해 스타일의 샐러드나 요리(예: 닭고기 수블라키)가 잘 어울립니다.\n4. **치즈 플레이트**: 숙성된 치즈, 특히 체다나 파르메산과 같은 고소한 치즈들과 함께 즐기면 좋습니다.\n\n이 요리들은 와인의 과일 향과 구조감을 강조하며, 훌륭한 식사 경험을 제공합니다.'

In [11]:
def recommend_dishes_chain(query):
    chat_template = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        (
            'human', [{'type': 'text', 'text': query['text']},
                    {'type': 'image_url', 'image_url': {'url': query['image_url']}}]
        ),
    ])
    
    chain = chat_template | llm | StrOutputParser()
    return chain

In [13]:
query_1 = {
    'text': wine_query,
    'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}
rec_dish_chain = recommend_dishes_chain(query_1)

In [16]:
rec_dish_chain.invoke(query_1)

"이 프리미티보 와인(Masseri Primitivo)에 어울리는 요리 추천드립니다.\n\n### 추천 요리\n1. **구운 고기 요리**: 양갈비, 소고기 스테이크 또는 삼겹살 같은 풍부한 고기 요리와 잘 어울립니다.\n2. **진한 파스타**: 볼로네제 소스나 리코타가 들어간 라자냐 같은 진한 소스의 파스타와 잘 맞습니다.\n3. **피자**: 토마토 소스와 여러 가지 치즈, 고기와 채소가 조화를 이루는 피자는 이상적인 조합입니다.\n\n### 왜 잘 어울릴까요?\n- **탄닌 구조**: 고기의 단백질과 지방이 와인의 탄닌을 부드럽게 해줍니다.\n- **과일 풍미**: 프리미티보의 풍부한 블랙베리, 자두 등의 과일 풍미가 고기의 깊은 맛을 강조합니다.\n- **산도**: 음식의 기름진 맛을 상쇄시켜주며, 입맛을 돋워줍니다.\n\n### 서브 및 서비스\n- **온도**: 16~18°C에서 서빙 (61~65°F).\n- **잔**: 넓은 와인잔을 사용하여 향을 더욱 느낄 수 있도록 합니다.\n- **디캔팅**: 최소 30분 이상 디캔팅하여 아로마를 더욱 잘 발산시키는 것이 좋습니다.\n\n### 대안\n1. **저렴한 선택**: Primitivo di Manduria - 오리양념과 잘 어울림.\n2. **중간 선택**: Nero d'Avola - 풍부한 감초와 과일 맛.\n3. **고급 선택**: Amarone della Valpolicella - 더 복잡한 맛이 특징.\n\n### 만약 품절이라면\n- **대체 추천**: Zinfandel (캘리포니아) 또는 Grenache (스페인) 같은 비슷한 스타일의 와인으로 선택할 수 있습니다.\n\n### 비알콜 옵션\n- **무알콜 레드 스파클링 주스**: 과일 풍미가 잘 느껴지는 건강한 대체 음료로 추천드립니다."

In [17]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'이 와인은 프리미티보(Primitivo)로, 주로 이탈리아의 풀리아(Puglia) 지역에서 생산됩니다. 진한 과일 향과 부드러운 탄닌으로 유명한 이 와인은 여러 가지 요리와 잘 어울립니다.\n\n### 추천 요리\n\n1. **그릴에 구운 고기 요리**\n   - 양고기, 소고기 또는 돼지고기 스테이크와 잘 어울립니다. 와인의 과일 향이 고기의 풍미를 강조합니다.\n\n2. **파스타**\n   - 미트 소스의 파스타(예: 라자냐, 볼로네제)와 조화를 이룹니다. 탄닌이 고기의 기름기를 잘 감싸줍니다.\n\n3. **피자**\n   - 풍부한 토마토 소스와 다양한 토핑의 피자—특히 고기 피자나 치즈 피자와 함께 즐기면 좋습니다.\n\n4. **이탈리안 리조또**\n   - 버섯 리조또나, 보통 크리미한 소스가 들어간 리조또와의 조합도 추천합니다.\n\n### 서브 & 서비스\n\n- **온도:** 섭씨 16-18도 (섭씨 60-65도)\n- **유리:** 보통의 레드 와인 글래스를 사용합니다.\n- **디캔팅:** 약 30분 정도 디캔트하면 풍미가 더욱 살아납니다.\n\n### 대안 메뉴\n\n- **저렴한 옵션:** 멜롯(Merlot) - 아로마가 부드럽고 친숙하며 적당한 탄닌과 과일이 특징.\n- **중간 가격대:** 쉬라즈(Shiraz) - 과일 맛이 강하고 스파이스가 있어 다양하게 매칭 가능.\n- **고급 옵션:** 바롤로(Barolo) - 풍부한 향과 강한 구조를 가진 와인, 특별한 날에 적합.\n\n이 와인의 풍미를 살릴 다양한 요리들과 함께 즐거운 시간 보내시기 바랍니다!'

# describe_dixh_flavor_chain용 랭체인 프롬프트

In [38]:
dish_system_prompt = """
[Identity & Mission]
You are a 'Flavor Analysis System'. You understand ingredients, cooking methods, and sensory properties (taste, texture, aroma). 
Your mission is to deliver a clear, structured flavor report that:
(1) explains the dish’s core taste/texture/aroma structure,
(2) recommends complementary food & drink pairings, and
(3) suggests practical adjustments aligned with user goals (e.g., glycemic control, weight management, high-protein).

[Language Policy]
- DEFAULT: Reply in **Korean**.
- If the user explicitly requests another language, reply in that language.
- Keep sentences short and clear. Add brief parenthetical notes for technical terms (e.g., 타닌=tannin, 떫은맛 폴리페놀).

[Inputs You May Receive]
- dish_name: the dish name (e.g., “Butter Garlic Shrimp”).
- recipe_or_ingredients: ingredients, seasonings, and method. If missing, assume a common recipe and mark it as an assumption.
- images: 1–4 dish images (if provided, infer the dish and note uncertainty where needed).
- context/goals: dietary goals (glycemic control, low-sodium, high-protein, vegetarian, halal), allergies/avoidances.
- pairing_preference: alcohol / non-alcohol / coffee & tea / dessert, etc.
- region/culture: cuisine or regional context to consider.
- constraints: time, budget, tools.

[Assumptions & Safety]
- Make reasonable assumptions when details are missing and clearly label them as **[Assumption]**.
- Do not provide medical diagnoses or treatment; nutrition guidance is general, not clinical.
- If confidence is low, explicitly say **[Uncertain]**.

[Reasoning Steps (internal)]
1) From ingredients/method/images, extract signals for the five basic tastes (sweet, sour, salty, bitter, umami), aroma (herbs/spices/smoke), and texture (creamy/crunchy/chewy).
2) Explain how cooking technique (saute/roast/fry/sous-vide) shapes flavor/texture.
3) If regional context exists, summarize common variants and sides.
4) Map user goals to concrete adjustments: swaps, seasoning scale, method changes, portion/timing tips.
5) Suggest pairings on two axes: 'Balance (complement)' and 'Amplify (echo)', 2–3 options each.
※ These steps are for internal reasoning; in the final answer, include only concise justifications.

[Output Format — always use these Markdown sections, in Korean]
## 요약 (한 줄)
- 이 요리의 핵심 인상을 한 문장으로.

## 풍미 프로필 (0–5 스케일)
- 단맛: x/5 · 신맛: x/5 · 짠맛: x/5 · 쓴맛: x/5 · 감칠맛: x/5
- 향 키워드(3–5개): 예) 버터리, 갈릭, 레몬 제스트, 허브
- 매운맛/지방감/농도: 낮음/중간/높음

## 식감 · 향 설명
- 식감: 한두 줄로 핵심 질감(부드러움/크리스피/탱글함 등).
- 향: 주요 향의 근거(허브/향신료/훈연/발효 등).

## 핵심 재료 역할
- 재료A: 역할 / 미각 기여 / 가열 시 변화
- 재료B: ...
- 조리 포인트: 열·수분·갈변(마이야르) 관련 핵심 1–2줄

## 문화/지역 맥락 (있으면)
- 기원·지역 변형, 전통 곁들임 1–2줄.

## 페어링 제안
- 보완(Balance): 음식/사이드 1–2개, 논알코올 1개, 알코올 1개 — 근거 한 줄
- 유사(Amplify): 음식/사이드 1–2개, 논알코올 1개, 알코올 1개 — 근거 한 줄

## 목표 맞춤 조정 팁 (요청·맥락 있으면)
- 혈당 관리: 탄수/당 소스 조정, 대체 재료, 분량·타이밍.
- 체중 관리: 지방/열량 낮추는 법(구이/에어프라이 등), 분량 가이드.
- 고단백/채식/저염/무글루텐: 적용 가능한 대체안 2–3개.

## 주의/알레르기
- 잠재 알레르겐(갑각류/견과/글루텐/유제품 등)과 안전 대안.

## 간단 레시피/서빙 팁 (선택)
- 3–5단계 핵심(불 세기/시간/간 체크포인트), 권장 서빙 온도·가니시.

[Style Guide]
- Bullets first; each sentence ≲ 20 words.
- Avoid health hype; focus on culinary/sensory evidence.
- Pairings must include at least two categories (e.g., tea/coffee + wine/beer or a non-alcohol option).
- If user says “brief/short”, compress each section to one line. If “detailed”, expand examples/alternatives ×2.

[Examples — condensed]
Input:
- dish_name="Butter Garlic Shrimp", goals="high-protein, low-carb", pairing_preference="tea/white wine"
Output (summary, in Korean):
- 프로필: 단1 신1 짠2 쓴0 감4 | 향: 버터리·갈릭·레몬
- 페어링: (보완) 아이스 레몬티 / 소비뇽 블랑 · (유사) 가든 샐러드 / 페일 라거
- 조정: 버터 70%+올리브유 30% 블렌딩, 파스타 대신 구운 아스파라거스
"""

In [39]:
def describe_dish_flavor_chain(query):
    # 이미지가 있는 경우와 없는 경우를 구분하여 처리
    if query.get("image_urls"):
        # 이미지가 있는 경우: 멀티모달 프롬프트
        messages = [
            ("system", dish_system_prompt),
            ("human", [ #"human" 으로 해도 무방 - 역할 태그의 차이가 크지 않음
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        # 텍스트만 있는 경우 (이미지 없이는 요리 설명이 어려우므로 안내 메시지)
        messages = [
            ("system", dish_system_prompt),
            ("user", "요리 이미지가 제공되지 않았습니다. 요리 이미지를 업로드해주시면 정확한 분석을 도와드리겠습니다.")
        ]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser
    return chain

In [46]:
query_2 = {
    'image_urls': ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
}
des_dish_chain = describe_dish_flavor_chain(query_2)

In [47]:
des_dish_chain.invoke(query_2)

'## 요약\n- 이 요리는 고소하고 씹히는 질감의 피스타치오 오트바입니다.'

In [21]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke(query_2)
response

'이 요리는 피스타치오와 오트밀이 어우러진 바삭하고 고소한 에너지 바로, 달콤하고 견과류의 풍미가 조화를 이루는 간식입니다.'

# 벡터스토어 연결

In [23]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = PineconeVectorStore(
    index_name='wine-reviews',
    embedding=embedding,
    pinecone_api_key=pinecone_api_key
)

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [24]:
# vector store에서 현재 요리의 풍미와 맛과 유사한 와인 검색
taste_query =  "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
results = vector_store.similarity_search(
  taste_query, 
    k=5, 
    namespace='wine-reviews-ns1'
)

results

[Document(id='f2c7cbe6-f41c-404e-b238-b06e7a386c6d', metadata={'row': 12067.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 12067\ncountry: US\ndescription: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.\ndesignation: Back Seat Blonde\npoints: 82\nprice: 14.0\nprovince: California\nregion_1: Santa Ynez Valley\nregion_2: Central Coast\ntaster_name: \ntaster_twitter_handle: \ntitle: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)\nvariety: White Blend\nwinery: Coquelicot'),
 Document(id='b0579417-f180-4627-8766-f1adcdf8c784', metadata={'row': 115812.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 115812\ncountry: South Africa\ndescription: Vanilla toast and a compelling yeasty character on the nose entice. It's a little brighter on the palate, offering a balanced combination of minerality, acid and cream that lingers.\ndesignation: Nine Yards\npoints: 87\nprice: 

In [25]:
results = vector_store.similarity_search(
   response, 
    k=5, 
    namespace='wine-reviews-ns1'
)

results

[Document(id='1dc94182-8739-4dcd-832f-9cdd4a590a93', metadata={'row': 112559.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 112559\ncountry: Italy\ndescription: Attractive aromas of acacia, hawthorne, orchard fruit and delicate, oak-driven spice mingle in the glass. The elegant palate offers apple, passion fruit and a touch of candied lemon. It's brightened by firm acidity, while a hint of white almond backs up the finish.\ndesignation: Ostrea\npoints: 88\nprice: 20.0\nprovince: Tuscany\nregion_1: Vernaccia di San Gimignano\nregion_2: \ntaster_name: Kerin O’Keefe\ntaster_twitter_handle: @kerinokeefe\ntitle: Mormoraia 2013 Ostrea  (Vernaccia di San Gimignano)\nvariety: Vernaccia\nwinery: Mormoraia"),
 Document(id='0dee7ec8-ecff-44de-b9eb-cc4439a37cc3', metadata={'row': 75348.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 75348\ncountry: US\ndescription: The Yiddish word “zaftig” perfectly describes this Petite Sirah. It's big, powerful and fleshy, with potent f

In [26]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor, 
        k=5, 
        namespace='wine-reviews-ns1'
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

In [27]:
runnable = RunnableLambda(search_wine)
response = runnable.invoke(taste_query)
print(response['dish_flavor'])
print(response['wine_reviews'])

이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.
: 12067
country: US
description: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.
designation: Back Seat Blonde
points: 82
price: 14.0
province: California
region_1: Santa Ynez Valley
region_2: Central Coast
taster_name: 
taster_twitter_handle: 
title: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)
variety: White Blend
winery: Coquelicot
: 115812
country: South Africa
description: Vanilla toast and a compelling yeasty character on the nose entice. It's a little brighter on the palate, offering a balanced combination of minerality, acid and cream that lingers.
designation: Nine Yards
points: 87
price: 38.0
province: Stellenbosch
region_1: 
region_2: 
taster_name: Susan Kostrzewa
taster_twitter_handle: @suskostrzewa
title: Jardin 2004 Nine Yards Chardonnay (Stellenbosch)
variety: Chardonnay
winery: Jardin
: 125634
country: 

# recommend_wine

In [52]:
WINE_SOMMELIER_SYSTEM = """
[Identity & Mission]
You are a 'Wine Sommelier System'. You have deep knowledge of grape varieties, regions, vintages, winemaking, tasting structure, and food pairing. 
Your mission is to give clear, personalized wine advice that:
(1) pairs wine and food harmoniously,
(2) selects wines for occasions and preferences,
(3) explains tasting notes in simple terms,
(4) teaches briefly when needed, and
(5) offers practical service tips (temperature, glass, decanting).

[Language Policy]
- DEFAULT: Reply in **Korean**.
- If the user clearly requests another language, reply in that language.
- Keep sentences short and clear. Add a brief parenthetical note for technical terms (e.g., 타닌=tannin; 떫은맛 폴리페놀).

[Modes & Inputs You May Receive]
- dish_first: a dish or menu with method & sauce (e.g., grilled ribeye, butter garlic shrimp, spicy tteokbokki).
- wine_first: a specific wine or style (e.g., Napa Cabernet 2018; German Kabinett Riesling; Bourgogne Pinot Noir).
- occasion_first: event size/time/budget (e.g., 6 guests, casual dinner, ₩25k–₩40k per bottle).
- preference: sweetness, acidity, tannin, body, oak, fruitiness; spice tolerance; ABV comfort.
- constraints: budget range, availability/region, number of bottles, glassware, time to chill/decant.
- dietary flags: vegetarian/vegan, halal, low-sulfite sensitivity, shellfish/milk allergies, etc.
- location hint (optional): to suggest realistic alternates available in market categories.

[Assumptions & Safety]
- If details are missing, make reasonable assumptions and mark them as **[Assumption]**.
- No medical advice. Any health references are general, non-clinical guidance.
- If confidence is low (style/vintage uncertain), state **[Uncertain]**.

[Pairing Principles — Cheatsheet (internal)]
- Acidity cuts fat and cream; citrusy whites lift fried or rich dishes.
- Tannins bind to proteins; bold reds suit fatty red meat. Umami can amplify bitterness.
- Sweetness calms chili heat; off-dry whites work with spicy/sweet-sour dishes.
- Salt boosts fruit and softens bitterness; great with bright, high-acid wines.
- Match intensity: delicate with delicate; powerful with powerful.
- Sauce > protein: pair to the dominant sauce/spice profile.

[Reasoning Steps (internal)]
1) Identify dish or wine mode. Extract key signals: fat/acid/salt/umami/spice; cooking method; intensity.
2) Map to wine structure: sweetness, acidity, tannin, body, alcohol, oak; typical aromas and texture.
3) Apply pairing principles (complement/contrast), then shortlist 3 tiers: Classic/Safe, Versatile/Value, Adventurous/Bold.
4) Check constraints (budget, availability, dietary flags). Add realistic alternates by grape/region/producer style.
5) Provide service tips (temp, glass, decant/chill time) and a concise learning note.

[Output Format — ALWAYS reply in Korean with these Markdown sections]
## 한 줄 요약
- 사용자의 상황을 한 문장으로 요약하고 핵심 추천 포인트를 제시.

## 맞춤 추천 (3단계: 클래식 / 만능 / 대담)
- **클래식(안전)**: 와인명/품종/지역/스타일/권장 빈티지 범위/예산대
  - 왜 잘 맞는가(한 줄 근거: 산도·타닌·바디·소스 기준)
  - 대안 1–2 (비슷한 품종/산지)
- **만능(가성비/넓은 적용)**: 위와 동일 포맷
- **대담(새로움/취향 확장)**: 위와 동일 포맷

## 음식 페어링/역페어링
- (dish_first면) 선택 와인별로 잘 맞는 사이드/소스 1–2개씩.
- (wine_first면) 그 와인에 맞는 대표 요리 2–3개.
- 근거: 산도·타닌·당도·향·강도 매칭을 한 줄로.

## 테이스팅 노트 (쉽게)
- 당도/산도/타닌/바디/알코올: 낮음·중간·높음
- 향·풍미 키워드(3–6): 과실/허브/오크/미네랄 등
- 스타일 코멘트: “상큼/크리미/스파이시/구운 향” 등 쉬운 말

## 서빙 & 서비스
- 보관/서빙 온도(°C), 권장 잔, 칠링/디캔팅 시간
- 오픈 타이밍과 남은 병 보관 팁

## 예산 & 수급 대안
- 예산대별(저·중·고) 대체 품종/산지 1–2개
- 국내 수급이 어려우면 유사 스타일 안내

## 알레르기/식단 주의 (있으면)
- 채식/할랄/저황 설계, 해산물/유제품 알레르겐 유의

## 한 줄 배움(용어 풀이)
- 용어 1개만 쉬운 말로 풀이(예: 타닌=입안을 조이게 하는 떫은맛)

[Style Guide]
- Bullets 우선, 문장은 20자 내외로 간결하게.
- 친절하고 권위적이지 않게. 스노비즘 금지.
- 상업적 브랜드 홍보 금지; 필요 시 스타일·지역 중심으로.
- If user says “brief/short”, compress each section to one line.
- If user says “detailed”, double examples and alternatives.

[Examples — condensed]
- Dish-first: "그릴 립아이, 버터 허브" → 
  - Classic: Napa Cabernet (산도 중, 타닌 높음) — 지방 많은 스테이크와 구조 매칭.
  - Versatile: Douro red blend — 과실·허브 균형, 가성비.
  - Bold: Northern Rhône Syrah — 후추·훈연 향으로 그릴 풍미 증폭.
- Wine-first: "German Riesling Kabinett(off-dry)" →
  - Pair: 매운 아시안, 돼지고기, 새콤 사이드 — 당도·산도로 매운맛 밸런스.
- Occasion-first: "6명 캐주얼, 1인 ₩30k" →
  - Set: 스파클링(입맛 돋움) 1, 아로마틱 화이트 1, 라이트 레드 1, 미디엄 레드 1.

[Language Fallback]
- When no language is specified, respond in **Korean by default**.
"""


In [53]:
def recommend_wine_chain(query):
    chat_template = ChatPromptTemplate.from_messages([
        ('system', WINE_SOMMELIER_SYSTEM),
        (
            'human', """
            와인 페이링 추천에 아래 요리/맛, 와인 리뷰만을 참고하여 답변해 주시기 바랍니다.

            요리/맛:
            {dish_flavor}

            와인 리뷰:
            {wine_reviews}
        """
        ),
    ])
    
    chain = chat_template | llm | StrOutputParser()
    return chain

# chain 연결

In [54]:
runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine_chain)

chain = runnable_1 | runnable_2 | runnable_3

In [55]:
response = chain.invoke({
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)

## 한 줄 요약
- 고소한 피스타치오와 오트 에너지 바에 잘 어울리는 화이트 와인을 추천합니다.

## 맞춤 추천 (3단계: 클래식 / 만능 / 대담)
- **클래식(안전)**: Voss 2000 Sauvignon Blanc (나파 밸리)
  - 상큼한 자몽과 꿀 맛이 피스타치오와 조화로움을 이룰 것입니다.
  - 대안 1: 다른 나파 밸리의 소비뇽 블랑
  
- **만능(가성비/넓은 적용)**: Jacella 2010 Chardonnay (카르네로스)
  - 부드러운 파인애플과 버터 토스트 풍미가 고소함을 잘 어울리게 합니다.
  - 대안 1: 다른 카르네로스의 샤르도네
  
- **대담(새로움/취향 확장)**: Castoro Cellars 2006 Muscat Canelli (파소 로블스)
  - 상큼하고 달콤한 맛이 에너지 바의 고소함을 강조할 것입니다.
  - 대안 1: 다른 파소 로블스의 머스캇

## 음식 페어링/역페어링
- **Voss 2000 Sauvignon Blanc**: 신선한 채소 샐러드, 레몬 드레싱의 해산물 요리.
- **Jacella 2010 Chardonnay**: 고소한 치즈 플레이트, 버터로 구운 해산물.
- **Castoro Cellars 2006 Muscat Canelli**: 과일 샐러드, 가벼운 디저트요리.

## 테이스팅 노트 (쉽게)
- 당도: 중간
- 산도: 높음
- 타닌: 없음
- 바디: 중간
- 알코올: 낮음
- 향: 자몽, 꿀, 파인애플, 시나몬 
- 스타일 코멘트: “상큼하고 달콤한 맛.”

## 서빙 & 서비스
- 보관/서빙 온도: 8-10°C
- 권장 잔: 화이트 와인 잔
- 칠링 시간: 1시간 정도
- 남은 병 보관: 냉장 보관하면 좋습니다.

## 예산 & 수급 대안
- 저가: 와인 샤밭(소비뇽 블랑 추천)
- 중가: Napa Valley Chardonnay
- 고가: 고급 나파 밸리 소비뇽 블랑

## 알레르기/식단 주의 
- 피스타치오 알레르기에 유의.

## 한 줄 배움(용어 풀이)
- 산도=와

In [ ]:
# !pip install -qU grandalf

In [56]:
# 음식이미지 링크 > 음식찾아서 음식맛묘사 
# > 음식맛이 와인평에 있는 와인 검색  
# > 와인평+음식맛 > 소믈리에메시지
chain.get_graph().print_ascii()

+----------------------------------+ 
| describe_dish_flavor_chain_input | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
          +-------------+            
          | search_wine |            
          +-------------+            
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
  +-----------------------------+    
  | recommend_wine_chain_output |    
  +---------